Романовский А.Н. 11.01.2023

    построение ассоциативных правил для датасета:
    Можно использовать например apriori, association_rules, fpgrowth из mlxtend.frequent_patterns (работают чуть быстрее)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('dataset.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose,dinner rolls,shampoo,all- purpose
1,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
2,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
3,juice,lunch meat,soda,toilet paper,all- purpose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs


_____
### Ознакомительная информация

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1499 non-null   object
 1   1       1499 non-null   object
 2   2       1499 non-null   object
 3   3       1499 non-null   object
 4   4       1499 non-null   object
 5   5       1448 non-null   object
 6   6       1411 non-null   object
 7   7       1356 non-null   object
 8   8       1300 non-null   object
 9   9       1241 non-null   object
 10  10      1203 non-null   object
 11  11      1156 non-null   object
 12  12      1103 non-null   object
 13  13      1044 non-null   object
dtypes: object(14)
memory usage: 164.1+ KB


In [5]:
df.isna().sum()

0       0
1       0
2       0
3       0
4       0
5      51
6      88
7     143
8     199
9     258
10    296
11    343
12    396
13    455
dtype: int64

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,1499,1499,1499,1499,1499,1448,1411,1356,1300,1241,1203,1156,1103,1044
unique,38,38,38,38,39,39,39,39,39,39,39,39,39,39
top,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables,vegetables
freq,108,98,95,140,113,97,95,103,89,98,78,90,77,81


### Предобработка

In [7]:
new = df.fillna('') #заменим пропуски

In [8]:
new = new.agg(', '. join , axis= 1).reset_index(drop=True) #обьединим позиции покупок в чек

In [9]:
new

0        pork,  sandwich bags,  lunch meat,  all- purp...
1        shampoo,  hand soap,  waffles,  vegetables,  ...
2        pork,  soap,  ice cream,  toilet paper,  dinn...
3        juice,  lunch meat,  soda,  toilet paper,  al...
4        pasta,  tortillas,  mixes,  hand soap,  toile...
                              ...                        
1494     beef,  sandwich bags,  hand soap,  paper towe...
1495     dinner rolls,  lunch meat,  spaghetti sauce, ...
1496     lunch meat,  eggs,  poultry,  vegetables,  to...
1497     ketchup,  milk,  poultry,  cheeses,  soap,  t...
1498     laundry detergent,  vegetables,  shampoo,  ve...
Length: 1499, dtype: object

In [18]:
type(new)

pandas.core.series.Series

In [14]:
rezult = new.to_frame()

In [15]:
rezult = rezult[0].transform(lambda x: x.split(", "))

In [16]:
rezult

0       [ pork,  sandwich bags,  lunch meat,  all- pur...
1       [ shampoo,  hand soap,  waffles,  vegetables, ...
2       [ pork,  soap,  ice cream,  toilet paper,  din...
3       [ juice,  lunch meat,  soda,  toilet paper,  a...
4       [ pasta,  tortillas,  mixes,  hand soap,  toil...
                              ...                        
1494    [ beef,  sandwich bags,  hand soap,  paper tow...
1495    [ dinner rolls,  lunch meat,  spaghetti sauce,...
1496    [ lunch meat,  eggs,  poultry,  vegetables,  t...
1497    [ ketchup,  milk,  poultry,  cheeses,  soap,  ...
1498    [ laundry detergent,  vegetables,  shampoo,  v...
Name: 0, Length: 1499, dtype: object

### Импорт библиотек и кодиорвка 

In [20]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [21]:
encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(rezult).transform(rezult), columns=encoder.columns_)
transactions.head(2)

,,,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,...,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt
0,False,False,True,True,False,True,True,False,False,False,...,True,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,True,True,False


In [22]:
transactions.columns

Index(['', ' ', ' all- purpose', ' aluminum foil', ' bagels', ' beef',
       ' butter', ' cereals', ' cheeses', ' coffee/tea', ' dinner rolls',
       ' dishwashing liquid/detergent', ' eggs', ' flour', ' fruits',
       ' hand soap', ' ice cream', ' individual meals', ' juice', ' ketchup',
       ' laundry detergent', ' lunch meat', ' milk', ' mixes', ' paper towels',
       ' pasta', ' pork', ' poultry', ' sandwich bags', ' sandwich loaves',
       ' shampoo', ' soap', ' soda', ' spaghetti sauce', ' sugar',
       ' toilet paper', ' tortillas', ' vegetables', ' waffles', ' yogurt'],
      dtype='object')

In [23]:
transactions[''].value_counts()

False    1044
True      455
Name: , dtype: int64

In [24]:
transactions.drop(['', ' '], axis=1, inplace = True) #удалим пустые столбцы

In [25]:
transactions.head(2)

,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,dinner rolls,dishwashing liquid/detergent,...,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt
0,True,True,False,True,True,False,False,False,True,False,...,True,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,True,True,False


In [27]:
from mlxtend.frequent_patterns import association_rules

Задаю минимальную поддержку в размере 6 из общего числа покупок. Значение Lift 1.5. Максимальное число товаров 3 

In [28]:
%%time
frequent_itemsets = apriori(transactions, min_support= 6/len(rezult), use_colnames=True, max_len =3)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1.5)
display(rules.head(20))
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"( tortillas, aluminum foil)",( sandwich loaves),0.058706,0.248833,0.022015,0.375000,1.507038,0.007407,1.201868
1,( sandwich loaves),"( tortillas, aluminum foil)",0.248833,0.058706,0.022015,0.088472,1.507038,0.007407,1.032655
2,"( cheeses, beef)",( sandwich bags),0.066711,0.250167,0.025350,0.380000,1.518987,0.008661,1.209408
3,( sandwich bags),"( cheeses, beef)",0.250167,0.066711,0.025350,0.101333,1.518987,0.008661,1.038526
4,"( mixes, beef)",( pork),0.065377,0.250167,0.025350,0.387755,1.549986,0.008995,1.224728
5,( pork),"( mixes, beef)",0.250167,0.065377,0.025350,0.101333,1.549986,0.008995,1.040011
6,"( sandwich loaves, cheeses)",( soap),0.064710,0.265510,0.026017,0.402062,1.514298,0.008836,1.228371
7,( soap),"( sandwich loaves, cheeses)",0.265510,0.064710,0.026017,0.097990,1.514298,0.008836,1.036896
8,"( dishwashing liquid/detergent, spaghetti sauce)",( fruits),0.061374,0.263509,0.025350,0.413043,1.567474,0.009178,1.254762
9,( fruits),"( dishwashing liquid/detergent, spaghetti sauce)",0.263509,0.061374,0.025350,0.096203,1.567474,0.009178,1.038535


Rules identified:  22
CPU times: total: 328 ms
Wall time: 316 ms


Ознакомимся с ассоциотивной связью позиции Pork, в чеках. 

In [29]:
pork_rules = rules[rules['consequents'].astype(str).str.contains('pork')]
pork_rules = pork_rules.sort_values(by=['lift'],ascending = [False]).reset_index(drop = True)

display(pork_rules.head(20))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"( mixes, beef)",( pork),0.065377,0.250167,0.025350,0.387755,1.549986,0.008995,1.224728
1,( juice),"( sandwich loaves, pork)",0.257505,0.060707,0.024016,0.093264,1.536298,0.008384,1.035906


____
### Алгоритм fpgrowth

Для данного алгоритма, установим такие же настройки 

In [30]:
from mlxtend.frequent_patterns import fpgrowth

In [31]:
%%time
frequent_itemsets_fpgrowth=fpgrowth(transactions, min_support= 6/len(rezult), use_colnames=True, max_len = 3)
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="lift",  min_threshold = 1.5)
display(rules_fpgrowth.sort_values (by='lift', ascending=False).head(5))
print("Rules identified: {}".format(len(rules_fpgrowth)))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,"( yogurt, sandwich loaves)",( juice),0.064043,0.257505,0.026017,0.406250,1.577639,0.009526,1.250518
15,( juice),"( yogurt, sandwich loaves)",0.257505,0.064043,0.026017,0.101036,1.577639,0.009526,1.041151
6,"( dishwashing liquid/detergent, spaghetti sauce)",( fruits),0.061374,0.263509,0.025350,0.413043,1.567474,0.009178,1.254762
7,( fruits),"( dishwashing liquid/detergent, spaghetti sauce)",0.263509,0.061374,0.025350,0.096203,1.567474,0.009178,1.038535
4,"( mixes, beef)",( pork),0.065377,0.250167,0.025350,0.387755,1.549986,0.008995,1.224728


Rules identified: 22
CPU times: total: 3.12 s
Wall time: 3.13 s


#### Вывод:
* Время работы алгоритма apriori составило 316 ms
* Время работы алгоритма fpgrowth составило 3.13 s
* данные алгоритмы хорошо подходят для ознакомления покупательского поведения